<a href="https://colab.research.google.com/github/RK-05/Jeopardy-/blob/main/%E2%80%9CJeopardy!%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: JEOPARDY_CSV.csv        


In [ ]:
!pip install flair
from flair.embeddings import WordEmbeddings , StackedEmbeddings , DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings

In [ ]:
from keras.models import Sequential
from keras.layers import Dense ,Dropout , Conv2D,Conv1D , Flatten , Embedding , MaxPool2D , LSTM,GlobalMaxPooling1D, MaxPool1D
from keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau
from nltk import word_tokenize

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('JEOPARDY_CSV.csv')
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [ ]:
df.isnull().sum()

Show Number    0
 Air Date      0
 Round         0
 Category      0
 Value         0
 Question      0
 Answer        2
dtype: int64

In [ ]:
len(df[' Value'].unique())

150

In [ ]:
df[df[' Value']=='None']

,Show Number,Air Date,Round,Category,Value,Question,Answer
55,4680,2004-12-31,Final Jeopardy!,THE SOLAR SYSTEM,None,Objects that pass closer to the sun than Mercu...,Icarus
116,5957,2010-07-06,Final Jeopardy!,HISTORIC WOMEN,None,She was born in Virginia around 1596 & died in...,Pocahontas
174,3751,2000-12-18,Final Jeopardy!,SPORTS LEGENDS,None,If Joe DiMaggio's hitting streak had gone one ...,H.J. Heinz (Heinz 57 Varieties)
235,3673,2000-07-19,Final Jeopardy!,THE MAP OF EUROPE,None,"Bordering Italy, Austria, Hungary & Croatia, i...",Slovenia
296,4931,2006-02-06,Final Jeopardy!,FAMOUS SHIPS,None,"On December 27, 1831 it departed Plymouth, Eng...",the HMS Beagle
...,...,...,...,...,...,...,...
216686,3940,2001-10-19,Final Jeopardy!,MAJOR LEAGUE BASEBALL TEAM NAMES,None,This team received its name after an 1890 inci...,Pittsburgh Pirates
216746,6044,2010-12-16,Final Jeopardy!,SKYSCRAPERS,None,After a construction boom fueled by oil & gas ...,Moscow
216807,5070,2006-09-29,Final Jeopardy!,NATIONAL CAPITALS,None,"This city's website calls it ""the last divided...",Nicosia
216868,5195,2007-03-23,Final Jeopardy!,BESTSELLING AUTHORS,None,He had the year's bestselling novel a record 7...,John Grisham


In [ ]:
#df.drop(df[df[' Value']=='None'],inplace=True,axis=1)

In [ ]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [ ]:
df[' Value'] = df[' Value'].apply(lambda x: int(0) if x=='None' else int(x.replace('$','').replace(',','')))
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [ ]:
lis = df[' Value'].apply(lambda x:round(x,-(len(str(x))-1)))
len(lis.unique())

24

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
lis.head()

0    200
1    200
2    200
3    200
4    200
Name:  Value, dtype: int64

In [ ]:
from keras.utils import np_utils

In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(lis)
encoded_Y = encoder.transform(lis)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
embedding = TransformerDocumentEmbeddings('bert-base-uncased')

In [ ]:
from tqdm import tqdm

In [ ]:
emb=[]
for sent in tqdm(df[' Question']):
    sent=Sentence(sent)
    embedding.embed(sent)
    emb.append(sent.get_embedding().cpu().detach().numpy())

100%|██████████| 216930/216930 [53:34<00:00, 67.48it/s]


In [ ]:
emb_cat=[]
for sent in tqdm(df[' Category']):
    sent=Sentence(sent)
    embedding.embed(sent)
    emb_cat.append(sent.get_embedding().cpu().detach().numpy())

100%|██████████| 216930/216930 [54:36<00:00, 66.21it/s]


In [ ]:
embedding_df=pd.DataFrame(np.array(emb))

In [ ]:
embedding_df_cat=pd.DataFrame(np.array(emb_cat))

In [ ]:
 embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.341941,0.094924,-0.376334,0.029432,-0.276497,0.164733,0.723231,0.616560,-0.155425,-0.365753,0.181413,0.013561,-0.761738,1.123792,0.279687,-0.207188,-0.093344,0.032123,0.079666,0.199756,-0.064871,0.018790,-0.278525,-0.057766,-0.001796,-0.179324,-0.295141,-0.254966,-0.063662,-0.026565,-0.103428,0.128542,-0.170032,-0.138652,0.348651,-0.401197,0.128721,-0.105483,0.408821,0.332912,...,0.367435,-0.687445,0.098264,0.495160,0.023792,0.003927,0.663600,-0.113314,-0.379603,-0.344899,0.080211,0.237598,0.405306,-0.031805,0.058832,-0.411982,0.797835,0.381055,-0.308689,-0.496416,-0.245136,0.246067,0.060829,0.367223,-6.200170,-0.405509,-0.136438,-0.253661,0.753087,-0.171838,-0.129338,-0.308410,0.437201,-0.368672,0.362269,-0.233356,0.246225,-0.269037,0.796916,0.425276
1,-1.128507,-0.233719,-0.844783,-0.343999,-0.750437,0.255770,0.687694,0.636580,-0.419601,0.137816,-0.128318,0.116446,-0.386482,0.577727,0.169465,0.187967,-0.658935,0.520990,-0.092766,0.289116,-0.334736,-0.258896,0.155673,-0.081731,0.129470,-0.018318,-0.725917,-0.237355,-0.140279,0.308504,-0.082538,0.237471,0.087356,-0.327513,0.197785,-0.617056,0.242382,0.001840,0.207416,0.509254,...,0.722117,-0.383471,-0.241573,0.296122,-0.162941,-0.431656,0.025582,0.060476,-0.437279,-0.174386,-0.641511,0.301926,-0.190386,-0.015724,0.098245,0.041024,0.349340,0.399569,0.451256,0.261718,0.015345,-0.173037,0.771476,0.155778,-5.708577,-0.153236,-0.562304,-0.484892,0.295399,0.048172,0.531744,-0.195859,0.306699,0.312698,0.905621,0.004755,-0.589927,-0.033032,0.354681,0.643932
2,0.094619,-0.242064,0.576521,-0.387843,-0.065934,-0.913236,0.427831,1.181923,-0.598973,-0.566863,-0.442924,-0.349443,-0.958734,0.714954,-0.526836,-0.180196,0.648143,0.967059,0.108464,0.406331,-0.095781,-0.708966,0.242528,0.394304,0.116582,-0.466965,0.256210,0.072991,0.531975,-0.053678,-0.246165,-0.199117,0.299777,-0.505421,0.401915,-0.281864,-0.002000,-0.288159,0.200616,0.503714,...,1.290082,-0.954728,0.313304,0.404931,0.658690,0.517945,0.675591,0.098386,-0.859829,0.418262,-0.659302,-0.205339,-0.929649,-0.232026,0.401901,0.062108,0.200821,0.468511,-0.640829,-0.953557,-0.441081,-0.131470,0.259628,-0.662283,-3.744334,-0.110969,-0.758636,-0.403865,-0.478761,-0.355652,0.329184,-0.189930,0.249549,-0.775645,-0.193334,0.470124,0.055535,0.206574,0.151918,0.525866
3,-0.305038,0.029787,0.092972,-0.217257,-0.084313,-0.072321,0.398166,0.841287,0.023509,-0.228974,0.437275,-0.633354,0.304974,0.783546,-0.259387,-0.066777,-0.357454,-0.105244,0.238309,-0.404650,-0.297471,-1.005919,0.374824,0.347509,-0.083939,-0.038334,-0.033184,-0.365142,-0.189619,0.327533,-0.094010,0.201667,-0.381318,-0.277377,-0.085561,-0.126885,0.393007,-0.280727,-0.181641,0.411613,...,-0.091412,-0.416713,-0.428174,0.202310,0.251766,0.108755,0.273161,-0.299223,-0.308041,0.173473,0.170597,0.734981,0.102962,-0.124902,-0.012301,-0.255094,0.521741,-0.135504,0.069239,0.072513,0.161355,0.036282,0.458882,0.065597,-4.859316,-0.286842,0.089336,-0.134671,-0.264942,-0.085686,-0.198908,-0.025481,-0.006927,0.039658,0.070242,0.181556,-0.190494,-0.136517,0.300830,0.005864
4,-0.782578,-0.142274,-0.479551,-0.011190,0.344046,0.138570,0.558167,0.747103,-0.484472,0.118545,0.388098,0.180945,-0.196684,0.990437,0.223871,0.076892,-0.439313,0.119782,0.323696,-0.049668,-0.309857,0.103998,0.141345,-0.369422,0.026889,-0.217255,-0.449079,-0.117807,-0.235157,0.040084,-0.243649,0.338617,-0.661340,-0.169628,0.102309,-0.469218,0.744263,0.068087,0.153180,0.209375,...,0.693112,0.204081,-0.258195,0.294535,-0.457003,-0.282465,0.107284,-0.051363,0.044945,-0.257412,-0.288259,0.091015,0.377953,-0.340515,0.338883,-0.022783,0.148259,0.770075,-0.062061,-0.068802,-0.212018,-0.314953,0.454769,0.075039,-6.862163,-0.862197,-0.811817,-0.627586,0.1708

In [ ]:
final_df=pd.concat([embedding_df,embedding_df_cat],axis=1)

In [ ]:
final_df.shape

(216930, 1536)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(final_df,encoded_Y,test_size=0.25)

In [ ]:
x_train=np.array(x_train)
x_validation=np.array(x_test)

In [ ]:
#x_test=np.array(emb)
x_validation=x_test
x_validation.shape

(54233, 1536)

In [ ]:
y_train=np.array(y_train)
y_validation=np.array(y_test)
print(y_train.shape,y_validation.shape)

(162697,) (54233,)


In [ ]:
X_train = x_train.reshape((-1, 768*2, 1))
X_train.shape

(162697, 1536, 1)

In [ ]:
X_test = x_validation.reshape((-1, 768*2, 1))
X_test.shape

(54233, 1536, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(256, 5,activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(180, 5,activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(128, 5,activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(60, activation='relu'))
model.add(Dense(24, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,verbose=1)

Epoch 1/200
5085/5085 [==============================] - 69s 7ms/step - loss: 2.2371 - accuracy: 0.1883 - val_loss: 2.1739 - val_accuracy: 0.1949
Epoch 2/200
5085/5085 [==============================] - 32s 6ms/step - loss: 2.1790 - accuracy: 0.1946 - val_loss: 2.1775 - val_accuracy: 0.1949
Epoch 3/200
5085/5085 [==============================] - 31s 6ms/step - loss: 2.1773 - accuracy: 0.1945 - val_loss: 2.1707 - val_accuracy: 0.1949
Epoch 4/200
5085/5085 [==============================] - 32s 6ms/step - loss: 2.1748 - accuracy: 0.1956 - val_loss: 2.1696 - val_accuracy: 0.1949
Epoch 5/200
5085/5085 [==============================] - 32s 6ms/step - loss: 2.1718 - accuracy: 0.1944 - val_loss: 2.1697 - val_accuracy: 0.1949
Epoch 6/200
5085/5085 [==============================] - 31s 6ms/step - loss: 2.1710 - accuracy: 0.1944 - val_loss: 2.1735 - val_accuracy: 0.1949
Epoch 7/200
5085/5085 [==============================] - 32s 6ms/step - loss: 2.1683 - accuracy: 0.1955 - val_loss: 2.1699 -

In [ ]:
model.save('jeo.h5')

In [ ]:
df[' Question'][0]

"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory"

In [ ]:
df[' Category'][0]

'HISTORY'

In [ ]:
sent="For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory"
sent=Sentence(sent)
embedding.embed(sent)

[Sentence: "For the last 8 years of his life , Galileo was under house arrest for espousing this man 's theory"   [− Tokens: 20]]

In [ ]:
sent1="HISTORY"
sent1=Sentence(sent1)
embedding.embed(sent1)

[Sentence: "HISTORY"   [− Tokens: 1]]

In [ ]:
emb_q=pd.DataFrame(sent.get_embedding().cpu().detach().numpy().reshape((1,768)))
emb_c=pd.DataFrame(sent1.get_embedding().cpu().detach().numpy().reshape((1,768)))

In [ ]:
final = pd.concat([emb_q,emb_c],axis=1)
final = np.array(final)

In [ ]:
final=final.reshape((1,1536,-1))
final.shape

(1, 1536, 1)

In [ ]:
pre=model.predict(final)

In [ ]:
encoder.inverse_transform([((pre)[0]).argmax()])[0]

400